In [1]:
# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import Dash
from dash import html
from dash import jupyter_dash
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



from shelter_Updated import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
#try block tests whether read query passes, otherwise determines authorization fails and prompts again
while True:
    name = input("Please enter username")
    password = input("Please enter password:")
    db = AnimalShelter(name, password)
    try:
        df = pd.DataFrame.from_records(db.read({}))
    except:
        print("Invalid username or password.  Please try again.")
    else:
        break



#########################
# Dashboard Layout / View
#########################
app = Dash("Friend Finder")

#Load Company logo
image_filename = 'newlogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div([
    html.Center(html.B(html.H1('Friend Finder Dashboard by Alex Frankel'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
    ], style={'textAlign':'center'}),
    html.Hr(),
    html.Div([     
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Wilderness Rescue', 'value': 'wildy'},
                {'label': 'Disaster Rescue', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset'
        )

    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[
                             {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
                         ],
                         data=df.to_dict('records'),
                         row_selectable = "single",
                         selected_rows=[0],
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='graph-id2',
            className='col s12 m6',

            ),
        html.Div(
            id='graph-id3',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):

    #if statements determine which radio button option has been selected
    #updates data table to show only columns from query
        
        #Water Rescue dogs
        if filter_type == "water":
            df = pd.DataFrame.from_records(db.read({
                "animal_type" : "Dog",
                "breed" : {"$in" : ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"
                                    ]},
                "sex_upon_outcome" : "Intact Female",
                "age_upon_outcome_in_weeks" : {"$gte" : 26.0, "$lte" : 156.0}
            }))
        
        #Wilderness Rescue dogs
        elif filter_type == "wildy":
            df = pd.DataFrame.from_records(db.read({
                "animal_type" : "Dog",
                "breed" : {"$in" : ["German Shepherd","Alaskan Malamute","Old English Sheepdog",
                                    "Siberian Husky", "Rottweiler"
                                    ]},
                "sex_upon_outcome" : "Intact Male",
                "age_upon_outcome_in_weeks" : {"$gte" : 26.0, "$lte" : 156.0}
            }))
            
        #Disaster Rescue dogs    
        elif filter_type == "disaster":
            df = pd.DataFrame.from_records(db.read({
                "animal_type" : "Dog",
                "breed" : {"$in" : ["Doberman Pinscher","German Shepherd","Golden Retriever",
                                    "Bloodhound", "Rottweiler"
                                    ]},
                "sex_upon_outcome" : "Intact Male",
                "age_upon_outcome_in_weeks" : {"$gte" : 20.0, "$lte" : 300.0}
            }))
            
        #Default or Reset selected
        else:
            df = pd.DataFrame.from_records(db.read({}))
                
       
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
       
       
        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    
    #local variable initialized to our data set
    dff = pd.DataFrame.from_dict(viewData)
    
    #returns pie chart of breeds
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]

# Display the outcome types of animals based on quantity represented in
# the data table

@app.callback(
    Output('graph-id2', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    
    #local variable initialized to our data set
    dff = pd.DataFrame.from_dict(viewData)
    
    #returns pie chart of outcome types
    return [
        dcc.Graph(
            figure = px.pie(dff, names='outcome_type', title='Prevalence of Outcome Types')
        )
    ]

# Display a histogram chart of the animals' ages

@app.callback(
    Output('graph-id3', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    
    #local variable initialized to our data set
    dff = pd.DataFrame.from_dict(viewData)
    
    #returns pie chart of breeds
    return [
        dcc.Graph(            
            figure = px.histogram(dff, x='age_upon_outcome_in_weeks', title='Animal Ages Distribution')
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Columns were imported in a different order than the original version and had to be updated accordingly
            # Column 12 and 13 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 15 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,12],dff.iloc[row,13]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,15])
                ])
            ])
        ])
    ]

#Removes inline exceptions from user's view
jupyter_dash.inline_exceptions = False

#app.run_server was deprecated to app,run
#WE use jupter_mode external to run on a local webpage
#specified unique port
app.run(debug=True,jupyter_mode="external",port=8051)


Please enter username aacuser
Please enter password: SNHU1234


Dash app running on http://127.0.0.1:8051/
